<a href="https://colab.research.google.com/github/marco-montesdeoca/identity_matrix/blob/main/identity_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction <br>
The Smith-Waterman algorithm has been used in many applications regarding strings comparisons, or alignments. <br>
In this notebook, we will use the Emboss implementation **water** to compute pairwise comparisons of nucleotide sequences, and later summarize the results as an indentity matrix. <br>
Identity matrices are useful in bioinformatics as these can let us know which isolates or samples are more closely related, from an evolutionary point of view. In the example data shown above, we use virus sequences (file names contain NCBI accession numbers), but this notebook can be used to analyze genes from any organism.

### Install required packages

In [ ]:
# Install BioPython package
! pip install biopython

In [ ]:
# Install Emboss package 
! sudo apt-get install emboss

### Download example data

In [ ]:
! wget https://github.com/marco-montesdeoca/identity_matrix/blob/main/sequences.zip?raw=true
! unzip sequences.zip?raw=true

### Data Analysis

In [3]:
import os
from Bio import AlignIO
import pandas as pd

In [5]:
sequences_dir = os.listdir('./sequences')
! mkdir alignments_water

In [ ]:
# Initialize identity table as dictionary
identD = dict()
aln_name = './alignments_water/aln.water'
for i in range(len(sequences_dir)):
    seq1 = sequences_dir[i]
    for j in range(len(sequences_dir)):
        seq2 = sequences_dir[j]
        key1 = seq1 + '|' + seq2 
        key2 = seq2 + '|' + seq1
        if (key1 not in identD) and (key2 not in identD):
            seq1_dir = './sequences/' + seq1
            seq2_dir = './sequences/' + seq2
            ! water $seq1_dir $seq2_dir -outfile $aln_name -datafile EDNAFULL -gapopen 10 -gapextend 0.5
            aln = AlignIO.read(open(aln_name), "emboss")
            aln_idnt = aln.annotations['identity']
            aln_len = len(aln[0])
            idnt_per = (aln_idnt/aln_len) * 100
            print(f'% idt: {idnt_per}, seq1: {seq1}, seq2: {seq2}')
            identD[key1] = idnt_per
            identD[key2] = idnt_per
            ! rm $aln_name

In [7]:
# Store results as a dataframe
df = pd.DataFrame(index = sequences_dir, columns = sequences_dir)
for key in identD.keys():
    row = key.split('|')[0]
    column = key.split('|')[1]
    per_id = identD[key]
    per_id = "{:.2f}".format(per_id)
    df.loc[row, column] = per_id

In [8]:
# Show results
df

,kf051855.1.fasta,kf373260.1.fasta,eu271682.1.fasta,jn711095.1.fasta,kf051860.1.fasta,eu625680.1.fasta,kf051859.1.fasta,eu851043.1.fasta,kf051871.1.fasta,jn711094.1.fasta,kf051888.1.fasta,jq712975.1.fasta
kf051855.1.fasta,100.00,100.00,98.64,98.53,100.00,98.32,100.00,98.64,99.69,98.64,99.90,98.85
kf373260.1.fasta,100.00,100.00,98.64,98.53,100.00,98.32,100.00,98.64,99.69,98.64,99.90,98.85
eu271682.1.fasta,98.64,98.64,100.00,99.48,98.64,99.48,98.64,100.00,98.74,99.58,98.53,99.79
jn711095.1.fasta,98.53,98.53,99.48,100.00,98.53,98.95,98.53,99.48,98.43,99.90,98.43,99.69
kf051860.1.fasta,100.00,100.00,98.64,98.53,100.00,98.32,100.00,98.64,99.69,98.64,99.90,98.85
eu625680.1.fasta,98.32,98.32,99.48,98.95,98.32,100.00,98.32,99.48,98.22,99.06,98.22,99.27
kf051859.1.fasta,100.00,100.00,98.64,98.53,100.00,98.32,100.00,98.64,99.69,98.64,99.90,98.85
eu851043.1.fasta,98.64,98.64,100.00,99.48,98.64,99.48,98.64,100.00,98.74,99.58,98.53,99.79
kf051871.1.fasta,99.69,99.69,98.74,98.43,99.69,98.22,99.69,98.74,100.00,98.53,99.79,98.74
jn711094.1.fasta,98.64,98.64,99.58,99.90,98.64,99.06,98.64,99.58,98.53,100.00,98.53,99.79


In [9]:
# Export identity matrix as excel file
df.to_excel("Identity matrix.xlsx")  